In [1]:
from __future__ import annotations

import heapq
import math
import os
import re
import sys
import time
from collections import defaultdict, deque
from itertools import combinations
from typing import Dict, List, Tuple

import aocd
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from matplotlib.path import Path
from ortools.sat.python import cp_model
from scipy.ndimage import convolve
from shapely.geometry import Polygon, box
from tqdm.notebook import tqdm, trange
from z3 import Int, Optimize, sat

In [2]:
p = aocd.get_puzzle(year=2025, day=12)

In [3]:
p.examples[0].input_data

'0:\n###\n##.\n##.\n\n1:\n###\n##.\n.##\n\n2:\n.##\n###\n##.\n\n3:\n##.\n###\n##.\n\n4:\n###\n#..\n###\n\n5:\n###\n.#.\n###\n\n4x4: 0 0 0 0 2 0\n12x5: 1 0 1 0 2 2\n12x5: 1 0 1 0 3 2'

In [4]:
def get_data(test_data: bool = True):
    if test_data:
        data = p.examples[0].input_data
    else:
        data = p.input_data
    return data

In [5]:
def process_data(data):
    _shapes = data.split("\n\n")[:-1]
    _grids = data.split("\n\n")[-1]
    shapes = {}
    grids = []
    for s in _shapes:
        i = int(s.split(":")[0])
        _shape = s.split(":")[1].split("\n")[1:]
        _shape = np.array([[1 if c == "#" else 0 for c in row] for row in _shape])
        shapes[i] = _shape
    for s in _grids.split("\n"):
        i = tuple([int(match) for match in re.findall(r"\d+", s.split()[0])])
        r = [int(t) for t in s.split()[1:]]
        g = (i, r)
        grids.append(g)

    return shapes, grids

In [6]:
data = get_data(test_data=False)
pieces, grids = process_data(data)

In [7]:
res = 0
areas = [p.sum() for _, p in pieces.items()]

for size, counts in grids:
    total_size = size[0] * size[1]
    total_area = 0

    for ind, count in enumerate(counts):
        total_area += areas[ind] * count

    res += 1 if total_area < total_size else 0

print(res)

565


In [8]:
# Even more simple

In [9]:
sum(1 for size, counts in grids if (size[0] * size[1] // 9) >= sum(counts))

565

In [11]:
import numpy as np
from z3 import *
from typing import Dict, List, Tuple, Optional
import time

# Define the pieces
pieces = {
    0: np.array([[1, 1, 1],
                 [1, 1, 0],
                 [1, 0, 0]]),
    1: np.array([[1, 1, 0],
                 [0, 1, 1],
                 [0, 0, 1]]),
    2: np.array([[0, 1, 1],
                 [1, 1, 0],
                 [1, 1, 1]]),
    3: np.array([[1, 1, 1],
                 [0, 1, 1],
                 [0, 1, 1]]),
    4: np.array([[1, 1, 1],
                 [1, 0, 0],
                 [1, 1, 1]]),
    5: np.array([[1, 1, 1],
                 [0, 1, 0],
                 [1, 1, 1]])
}


def generate_orientations(piece: np.ndarray) -> List[np.ndarray]:
    """Generate all unique orientations of a piece"""
    orientations = []
    seen = set()
    
    for flip in [False]:
        current = np.flipud(piece) if flip else piece.copy()
        
        for rotation in range(4):
            rotated = np.rot90(current, rotation)
            as_tuple = tuple(map(tuple, rotated))
            
            if as_tuple not in seen:
                seen.add(as_tuple)
                orientations.append(rotated)
    
    return orientations


def get_piece_cells(piece: np.ndarray) -> List[Tuple[int, int]]:
    """Find coordinates of all 1's in a piece"""
    return [(int(i), int(j)) for i, j in zip(*np.where(piece == 1))]


# Global cache for placements
_placement_cache = {}

def generate_all_placements_cached(pieces: Dict, grid_height: int, grid_width: int, verbose: bool = False):
    """Generate all possible placements with caching"""
    cache_key = (grid_height, grid_width)
    
    if cache_key in _placement_cache:
        if verbose:
            print("Using cached placements")
        return _placement_cache[cache_key]
    
    all_placements = {piece_id: [] for piece_id in pieces.keys()}
    
    for piece_id, piece in pieces.items():
        if verbose:
            print(f"Generating placements for piece {piece_id}...")
        orientations = generate_orientations(piece)
        
        for oriented_piece in orientations:
            height, width = oriented_piece.shape
            cells = get_piece_cells(oriented_piece)
            
            for y in range(grid_height - height + 1):
                for x in range(grid_width - width + 1):
                    grid_cells = tuple((y + dy, x + dx) for dy, dx in cells)
                    all_placements[piece_id].append(grid_cells)
        
        if verbose:
            print(f"  Piece {piece_id}: {len(all_placements[piece_id])} possible placements")
    
    _placement_cache[cache_key] = all_placements
    return all_placements


def solve_puzzle_z3(pieces: Dict, 
                    piece_counts: List[int], 
                    grid_height: int, 
                    grid_width: int,
                    verbose: bool = False,
                    timeout: int = 60000) -> Tuple[Optional[np.ndarray], Optional[List]]:
    """
    Solve the puzzle using Z3 SMT solver.
    
    Args:
        pieces: Dictionary of piece shapes
        piece_counts: List of how many of each piece to place
        grid_height: Height of the grid
        grid_width: Width of the grid
        verbose: If True, print progress information
        timeout: Timeout in milliseconds (default 60 seconds)
    
    Returns:
        Tuple of (solution_grid, selected_placements) or (None, None) if no solution
    """
    if verbose:
        print("\n=== Z3 Solver ===")
        print(f"Grid: {grid_height}x{grid_width} = {grid_height * grid_width} cells")
        print(f"Pieces to place: {sum(piece_counts)} total")
    
    # Generate all possible placements
    if verbose:
        print("\nGenerating placements...")
    start_time = time.time()
    all_placements = generate_all_placements_cached(pieces, grid_height, grid_width, verbose)
    if verbose:
        print(f"Time: {time.time() - start_time:.2f}s")
    
    # Create Z3 solver
    if verbose:
        print("\nBuilding Z3 model...")
    
    solver = Solver()
    solver.set("timeout", timeout)
    
    # Create boolean variables for each possible placement
    placement_vars = {}
    for piece_id in pieces.keys():
        placement_vars[piece_id] = [
            Bool(f'p{piece_id}_{idx}') 
            for idx in range(len(all_placements[piece_id]))
        ]
    
    # Constraint 1: Exactly N placements of each piece type
    if verbose:
        print("  Adding piece count constraints...")
    for piece_id, count in enumerate(piece_counts):
        # Use PbEq for "exactly N" constraint (Pseudo-Boolean)
        solver.add(PbEq([(var, 1) for var in placement_vars[piece_id]], count))
    
    # Constraint 2: Each cell covered at most once
    if verbose:
        print("  Building cell coverage constraints...")
    
    cell_coverage = {}
    for piece_id in pieces.keys():
        for idx, placement_cells in enumerate(all_placements[piece_id]):
            var = placement_vars[piece_id][idx]
            for cell in placement_cells:
                if cell not in cell_coverage:
                    cell_coverage[cell] = []
                cell_coverage[cell].append(var)
    
    if verbose:
        print("  Adding cell coverage constraints...")
    # Add: each cell covered at most once
    for cell, vars_covering in cell_coverage.items():
        if len(vars_covering) > 1:
            # AtMost(vars, 1) means at most one variable can be True
            solver.add(AtMost(*vars_covering, 1))
    
    if verbose:
        total_vars = sum(len(v) for v in placement_vars.values())
        print(f"  Total variables: {total_vars}")
        print(f"  Total constraints: {len(pieces) + sum(1 for v in cell_coverage.values() if len(v) > 1)}")
    
    # Solve
    if verbose:
        print("\nSolving...")
    start_time = time.time()
    result = solver.check()
    solve_time = time.time() - start_time
    
    if verbose:
        print(f"\n=== Result ===")
        print(f"Status: {result}")
        print(f"Time: {solve_time:.2f}s")
    
    if result == sat:
        if verbose:
            print("✓ SOLUTION FOUND!")
        
        # Extract solution
        model = solver.model()
        solution_grid = np.zeros((grid_height, grid_width), dtype=np.int8)
        selected_placements = []
        
        for piece_id in pieces.keys():
            for idx, var in enumerate(placement_vars[piece_id]):
                if is_true(model[var]):
                    placement_cells = all_placements[piece_id][idx]
                    selected_placements.append((piece_id, placement_cells))
                    
                    for cell in placement_cells:
                        solution_grid[cell] = piece_id + 1
        
        if verbose:
            filled_cells = np.sum(solution_grid > 0)
            print(f"\nStatistics:")
            print(f"  Filled: {filled_cells}/{grid_height * grid_width}")
            print(f"  Empty: {grid_height * grid_width - filled_cells}")
        
        return solution_grid, selected_placements
    
    elif result == unsat:
        if verbose:
            print("✗ NO SOLUTION POSSIBLE")
        return None, None
    else:
        if verbose:
            print("? UNKNOWN (timeout or error)")
        return None, None


def solve_puzzle_z3_fast(pieces: Dict, 
                         piece_counts: List[int], 
                         grid_height: int, 
                         grid_width: int,
                         timeout: int = 30000) -> bool:
    """
    Fast Z3 version that only returns True/False for feasibility.
    Timeout in milliseconds (default 30 seconds).
    """
    all_placements = generate_all_placements_cached(pieces, grid_height, grid_width, False)
    
    solver = Solver()
    solver.set("timeout", timeout)
    
    # Create variables
    placement_vars = {}
    for piece_id in pieces.keys():
        placement_vars[piece_id] = [
            Bool(f'p{piece_id}_{idx}') 
            for idx in range(len(all_placements[piece_id]))
        ]
    
    # Piece count constraints
    for piece_id, count in enumerate(piece_counts):
        solver.add(PbEq([(var, 1) for var in placement_vars[piece_id]], count))
    
    # Cell coverage constraints
    cell_coverage = {}
    for piece_id in pieces.keys():
        for idx, placement_cells in enumerate(all_placements[piece_id]):
            var = placement_vars[piece_id][idx]
            for cell in placement_cells:
                if cell not in cell_coverage:
                    cell_coverage[cell] = []
                cell_coverage[cell].append(var)
    
    for vars_covering in cell_coverage.values():
        if len(vars_covering) > 1:
            solver.add(AtMost(*vars_covering, 1))
    
    # Solve
    result = solver.check()
    return result == sat


def batch_solve_puzzles_z3(puzzle_specs: List[Tuple[int, int, List[int]]], 
                           pieces: Dict,
                           timeout_per_puzzle: int = 30000,
                           verbose: bool = True) -> List[bool]:
    """
    Solve multiple puzzles efficiently with Z3.
    
    Args:
        puzzle_specs: List of (height, width, piece_counts) tuples
        pieces: Dictionary of piece shapes
        timeout_per_puzzle: Max time per puzzle in milliseconds
        verbose: Print progress
    
    Returns:
        List of boolean results
    """
    results = []
    
    if verbose:
        print(f"\n=== Batch solving {len(puzzle_specs)} puzzles with Z3 ===")
    
    start_time = time.time()
    
    for i, (height, width, counts) in enumerate(puzzle_specs):
        if verbose and i % 10 == 0:
            elapsed = time.time() - start_time
            avg_time = elapsed / (i + 1) if i > 0 else 0
            remaining = avg_time * (len(puzzle_specs) - i)
            print(f"Progress: {i}/{len(puzzle_specs)} | Avg: {avg_time:.2f}s | ETA: {remaining/60:.1f}min")
        
        result = solve_puzzle_z3_fast(pieces, counts, height, width, timeout_per_puzzle)
        results.append(result)
    
    if verbose:
        total_time = time.time() - start_time
        feasible_count = sum(results)
        print(f"\n=== Batch complete ===")
        print(f"Total time: {total_time/60:.1f} minutes")
        print(f"Avg per puzzle: {total_time/len(puzzle_specs):.2f}s")
        print(f"Feasible: {feasible_count}/{len(puzzle_specs)}")
    
    return results


# Example usage
if __name__ == "__main__":
    # Test single puzzle
    print("Testing Z3 solver:")
    GRID_HEIGHT = 44
    GRID_WIDTH = 41
    piece_counts = [54, 50, 44, 38, 45, 49]
    
    solution_grid, placements = solve_puzzle_z3(
        pieces, 
        piece_counts, 
        GRID_HEIGHT, 
        GRID_WIDTH,
        verbose=True,
        timeout=60000  # 60 seconds
    )
    
    if solution_grid is not None:
        print("\n✓ Solution found!")
        np.save('puzzle_solution_z3.npy', solution_grid)
    
    # Compare with fast version
    print("\n" + "="*60)
    print("\nTesting fast Z3 version:")
    
    start = time.time()
    result = solve_puzzle_z3_fast(pieces, piece_counts, GRID_HEIGHT, GRID_WIDTH, timeout=30000)
    print(f"Result: {'FEASIBLE' if result else 'INFEASIBLE'}")
    print(f"Time: {time.time() - start:.2f}s")

Testing Z3 solver:

=== Z3 Solver ===
Grid: 44x41 = 1804 cells
Pieces to place: 280 total

Generating placements...
Generating placements for piece 0...
  Piece 0: 6552 possible placements
Generating placements for piece 1...
  Piece 1: 6552 possible placements
Generating placements for piece 2...
  Piece 2: 6552 possible placements
Generating placements for piece 3...
  Piece 3: 6552 possible placements
Generating placements for piece 4...
  Piece 4: 6552 possible placements
Generating placements for piece 5...
  Piece 5: 3276 possible placements
Time: 0.03s

Building Z3 model...
  Adding piece count constraints...
  Building cell coverage constraints...
  Adding cell coverage constraints...
  Total variables: 36036
  Total constraints: 1810

Solving...

=== Result ===
Status: unknown
Time: 60.02s
? UNKNOWN (timeout or error)


Testing fast Z3 version:
Result: INFEASIBLE
Time: 32.81s
